In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import xgboost as xgb

In [2]:
from sklearn.metrics import mean_absolute_error
from datetime import date

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
# set the seed of random number generator, which is useful for creating simulations 
# or random objects that can be reproduced.
import random
SEED=3
random.seed(SEED)
np.random.seed(SEED)

In [5]:
# Load Train Data
train = pd.read_csv('../data/processed/train_aggr.csv', sep=';')

In [6]:
train.shape

(34540, 42)

In [7]:
train['fecha_venta_norm'] = pd.to_datetime(train['fecha_venta_norm'])

In [8]:
train['fecha_venta_norm'] = train['fecha_venta_norm'].dt.date

In [9]:
# Filtramos los meses que consideramos buenos para el entrenamiento (11 y 12)
train = train[train.fecha_venta_norm.isin([#date(2012, 11, 1),
                                                 date(2012, 12, 1),
                                                #date(2013, 11, 1), 
                                                 date(2013, 12, 1), 
                                                 #date(2014, 11, 1)
])]

In [10]:
predictors = ['id_pos','canal', 'competidores',
       'ingreso_mediana', 'densidad_poblacional',
       'pct_0a5', 'pct_5a9', 'pct_10a14', 'pct_15a19', 'pct_20a24',
       'pct_25a29', 'pct_30a34', 'pct_35a39', 'pct_40a44', 'pct_45a49',
       'pct_50a54', 'pct_55a59', 'pct_60a64', 'pct_65a69', 'pct_70a74',
       'pct_75a79', 'pct_80a84', 'pct_85ainf', 'pct_bachelors',
       'pct_doctorados', 'pct_secundario', 'pct_master', 'pct_bicicleta',
       'pct_omnibus', 'pct_subtes', 'pct_taxi', 'pct_caminata',
       'mediana_valor_hogar', 'unidades']

In [11]:
train = train[predictors]

#### encode catvars

In [12]:
le = preprocessing.LabelEncoder()

In [13]:
le.fit(train['canal'].unique())

LabelEncoder()

In [14]:
train['canal'] = le.transform(train['canal'].values)

In [15]:
X, y = train.iloc[:,:-1],train.iloc[:,-1]


In [16]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

### Hyperparameters tuning

In [18]:
## =========================================================================================================
# 
#  Booster Parameters 
#
# n_estimators
#    - The number of sequential trees to be modeled
#    - Though GBM is fairly robust at higher number of trees but it can still overfit at a point.
#
# max_depth [default=6]
#    - The maximum depth of a tree.
#    - Used to control over-fitting as higher depth will allow model to learn relations very pecific to a particular sample.
#    - Typical values: 3-10
#
# min_child_weight [default=1]
#    - Defines the minimum sum of weights of all observations required in a child.
#    - This is similar to min_child_leaf in GBM but not exactly.
#      This refers to min sum of weightsof observations while GBM has min number of observations.
#    - Used to control over-fitting. Higher values prevent a model from learning relations
#      which might be highly specific to the particular sample selected for a tree.
#    - Too high values can lead to under-fitting hence, it should be tuned using CV.
#
# eta : learning rate
#   - Makes the model more robust by shrinking the weights on each step
#   - Typical final values to be used: 0.01-0.2
#
# gamma [default=0]
#
#    - A node is split only when the resulting split gives a positive reduction in the loss function.
#    - Gamma specifies the minimum loss reduction required to make a split.
#    - Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.
#
# subsample [default=1]
#     - Denotes the fraction of observations to be randomly samples for each tree.
#     - Lower values make the algorithm more conservative and prevents overfitting but too small values
#       might lead to under-fitting.
#     - Typical values: 0.5-1
#
# colsample_bytree [default=1]
#     - Denotes the fraction of columns to be randomly samples for each tree.
#     - Typical values: 0.5-1
#
# alpha [default=0]
#     - L1 regularization term on weight (analogous to Lasso regression)
#     - Can be used in case of very high dimensionality so that the algorithm runs faster when implemented
#
# scale_pos_weight [default=1]
#     A value greater than 0 should be used in case of high class imbalance as it helps in faster convergence.
#
## ===========================================================================================================


#### Number of trees (estimators)

In [19]:
# 5 fold cross validation is more accurate than using a single validation set
cv_folds = 5
early_stopping_rounds = 50
model = xgb.XGBRegressor(seed = SEED)

In [20]:
xgb_param = model.get_xgb_params()

In [21]:
xgb_param

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'importance_type': 'gain',
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'nthread': 1,
 'objective': 'reg:linear',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 3,
 'subsample': 1,
 'verbosity': 1}

In [21]:
# To train on GPU
xgb_param['objective'] = 'reg:squarederror'
xgb_param['gpu_id'] = 0
xgb_param['max_bin'] = 16
xgb_param['tree_method'] = 'gpu_hist'
xgb_param['learning_rate'] = 0.01

In [22]:
cvresult = xgb.cv(params=xgb_param, dtrain=data_dmatrix, num_boost_round = 1000, nfold = cv_folds, metrics = 'mae', early_stopping_rounds = early_stopping_rounds, seed = SEED)
print(cvresult)
print ("Optimal number of trees (estimators) is %i" % cvresult.shape[0])

     train-mae-mean  train-mae-std  test-mae-mean  test-mae-std
0         26.915981       0.054026      26.916026      0.217875
1         26.647311       0.053501      26.647438      0.217424
2         26.381565       0.052956      26.381764      0.216905
3         26.119152       0.052414      26.119396      0.216394
4         25.861280       0.051861      25.861614      0.215815
5         25.606877       0.051326      25.607226      0.215276
6         25.356213       0.050859      25.356646      0.214734
7         25.108317       0.050328      25.108745      0.214277
8         24.863579       0.049815      24.864052      0.213767
9         24.621905       0.049360      24.622397      0.213322
10        24.383354       0.049013      24.383942      0.212760
11        24.148663       0.048543      24.149362      0.212216
12        23.917462       0.048028      23.918044      0.211386
13        23.689440       0.047471      23.689941      0.210870
14        23.466430       0.047039      

#### max_depth & min_child_weight

In [23]:
model.set_params(objective = 'reg:squarederror')
model.set_params(gpu_id = 0)
model.set_params(max_bin= 16)

model.set_params(learning_rate = 0.01)
model.set_params(n_estimators=194)
model.set_params(tree_method='gpu_hist')


param_test1 = {
'max_depth': [i for i in range(2,8,1)],
'min_child_weight': [i for i in range(1,6,1)]
}

In [24]:
gsearch1 = GridSearchCV(estimator = model, param_grid = param_test1, scoring = 'neg_mean_absolute_error',  iid = False, cv = cv_folds, verbose = 1)

In [25]:
res =gsearch1.fit(X_train,y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  4.2min finished


In [26]:
#print gsearch1.grid_scores_
print(gsearch1.best_params_)
print(gsearch1.best_score_)

{'max_depth': 7, 'min_child_weight': 2}
-12.903912042445237


#### gamma

In [27]:
model.set_params(max_depth = 7)
model.set_params(min_child_weight = 2)

param_test2 = {
'gamma':[i/10.0 for i in range(0,5)]
}


In [28]:
gsearch2 = GridSearchCV(estimator = model, param_grid = param_test2, scoring = 'neg_mean_absolute_error',  iid = False, cv = cv_folds, verbose = 1)
gsearch2.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   60.0s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
       importance_type='gain', learning_rate=0.01, max_bin=16,
       max_delta_step=0, max_depth=7, min_child_weight=2, missing=None,
       n_estimators=1...1, scale_pos_weight=1, seed=3, silent=None, subsample=1,
       tree_method='gpu_hist', verbosity=1),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=1)

In [29]:
print(gsearch2.best_params_)
print(gsearch2.best_score_)

{'gamma': 0.0}
-12.903912042445237


#### Recal number of trees (estimators)

In [18]:
# 5 fold cross validation is more accurate than using a single validation set
cv_folds = 5
early_stopping_rounds = 50
model = xgb.XGBRegressor(seed = SEED)

In [22]:
xgb_param = model.get_xgb_params()

In [22]:
# To train on GPU
xgb_param['objective'] = 'reg:squarederror'
xgb_param['gpu_id'] = 0
xgb_param['max_bin'] = 16
xgb_param['tree_method'] = 'gpu_hist'
xgb_param['learning_rate'] = 0.01
xgb_param['gamma'] = 0.0
xgb_param['max_depth'] = 7
xgb_param['min_child_weight'] = 2

In [23]:
cvresult = xgb.cv(params=xgb_param, dtrain=data_dmatrix, num_boost_round = 1000, nfold = cv_folds, metrics = 'mae', early_stopping_rounds = early_stopping_rounds, seed = SEED)
print(cvresult)
print ("Optimal number of trees (estimators) is %i" % cvresult.shape[0])

     train-mae-mean  train-mae-std  test-mae-mean  test-mae-std
0         26.917710       0.053999      26.917679      0.218485
1         26.650690       0.053441      26.650699      0.218459
2         26.386586       0.052918      26.386501      0.218486
3         26.125785       0.052432      26.125932      0.217811
4         25.869141       0.051933      25.869835      0.217685
5         25.616086       0.051461      25.616895      0.217627
6         25.366524       0.051067      25.367503      0.217386
7         25.119700       0.050521      25.121227      0.217174
8         24.876011       0.050075      24.877820      0.216975
9         24.635239       0.049586      24.637687      0.216456
10        24.397578       0.049246      24.400493      0.216270
11        24.163297       0.048797      24.166944      0.215834
12        23.932572       0.048398      23.937054      0.215733
13        23.705100       0.047991      23.710760      0.215795
14        23.481435       0.047523      

- N_estimators = 93 sigue siendo el mejor valor

#### subsample & colsample_bytree

In [25]:
model.set_params(objective = 'reg:squarederror')
model.set_params(gpu_id = 0)
model.set_params(max_bin= 16)
model.set_params(tree_method='gpu_hist')
model.set_params(learning_rate = 0.01)
model.set_params(n_estimators=209)
model.set_params(max_depth = 7)
model.set_params(min_child_weight = 2)
model.set_params(gamma = 0.0)



param_test3 = {
'subsample' : [i/10.0 for i in range(6,11)],
'colsample_bytree' : [i/10.0 for i in range(6,11)]
}


In [26]:
model.get_xgb_params

<bound method XGBModel.get_xgb_params of XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.0, gpu_id=0,
       importance_type='gain', learning_rate=0.01, max_bin=16,
       max_delta_step=0, max_depth=7, min_child_weight=2, missing=None,
       n_estimators=209, n_jobs=1, nthread=None,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=3, silent=None, subsample=1,
       tree_method='gpu_hist', verbosity=1)>

In [27]:
gsearch3 = GridSearchCV(estimator = model, param_grid = param_test3, scoring = 'neg_mean_absolute_error',  iid = False, cv = cv_folds, verbose = 1)
gsearch3.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:  5.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.0, gpu_id=0,
       importance_type='gain', learning_rate=0.01, max_bin=16,
       max_delta_step=0, max_depth=7, min_child_weight=2, missing=None,
       n_estimators...1, scale_pos_weight=1, seed=3, silent=None, subsample=1,
       tree_method='gpu_hist', verbosity=1),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'subsample': [0.6, 0.7, 0.8, 0.9, 1.0], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=1)

In [28]:
print(gsearch3.best_params_)
print(gsearch3.best_score_)

{'colsample_bytree': 1.0, 'subsample': 0.8}
-12.854480665766213


#### reg_alpha

In [29]:
model.set_params(objective = 'reg:squarederror')
model.set_params(gpu_id = 0)
model.set_params(max_bin= 16)
model.set_params(tree_method='gpu_hist')
model.set_params(learning_rate = 0.01)
model.set_params(n_estimators=209)
model.set_params(max_depth = 7)
model.set_params(min_child_weight = 2)
model.set_params(colsample_bytree = 1.0)
model.set_params(subsample = 0.8)
model.set_params(gamma = 0.0)

param_test4 = {
'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}



In [30]:
gsearch4 = GridSearchCV(estimator = model, param_grid = param_test4, scoring = 'neg_mean_absolute_error',  iid = False, cv = cv_folds, verbose = 1)
gsearch4.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1.0, gamma=0.0, gpu_id=0,
       importance_type='gain', learning_rate=0.01, max_bin=16,
       max_delta_step=0, max_depth=7, min_child_weight=2, missing=None,
       n_estimato... scale_pos_weight=1, seed=3, silent=None,
       subsample=0.8, tree_method='gpu_hist', verbosity=1),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'reg_alpha': [1e-05, 0.01, 0.1, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=1)

In [31]:
print(gsearch4.best_params_)
print(gsearch4.best_score_)

{'reg_alpha': 100}
-12.849600719385526


#### Training and Testing Model

In [32]:
model = xgb.XGBRegressor(seed = SEED)

In [33]:
model.set_params(objective = 'reg:squarederror')
model.set_params(gpu_id = 0)
model.set_params(max_bin= 16)
model.set_params(tree_method='gpu_hist')
model.set_params(learning_rate = 0.01)
model.set_params(n_estimators=209)
model.set_params(max_depth = 7)
model.set_params(min_child_weight = 2)
model.set_params(colsample_bytree = 1.0)
model.set_params(subsample = 0.8)
model.set_params(gamma = 0.0)
model.set_params(reg_alpha = 100)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1.0, gamma=0.0, gpu_id=0,
       importance_type='gain', learning_rate=0.01, max_bin=16,
       max_delta_step=0, max_depth=7, min_child_weight=2, missing=None,
       n_estimators=209, n_jobs=1, nthread=None,
       objective='reg:squarederror', random_state=0, reg_alpha=100,
       reg_lambda=1, scale_pos_weight=1, seed=3, silent=None,
       subsample=0.8, tree_method='gpu_hist', verbosity=1)

In [34]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1.0, gamma=0.0, gpu_id=0,
       importance_type='gain', learning_rate=0.01, max_bin=16,
       max_delta_step=0, max_depth=7, min_child_weight=2, missing=None,
       n_estimators=209, n_jobs=1, nthread=None,
       objective='reg:squarederror', random_state=0, reg_alpha=100,
       reg_lambda=1, scale_pos_weight=1, seed=3, silent=None,
       subsample=0.8, tree_method='gpu_hist', verbosity=1)

In [35]:
y_pred = model.predict(X_test)

In [36]:
print("MAE unidades: ",mean_absolute_error(y_test, y_pred))

MAE unidades:  13.004424819618848


In [37]:
print("mean unidades pred: ", np.mean(y_pred))

mean unidades pred:  23.99622


In [38]:
print("median unidades pred: ", np.median(y_pred))

median unidades pred:  20.809826
